README

В процессе написание кода программе нехватало выделенной памяти. На момент последнего запкуска на моей машине ошибки были устранены, надеюсь, у Вас всё будет работать :)
Для повторения представленных результатов достаточно запустить последовательное исполнение всех ячеек.

# Финальное задание

## Подход №1. Градиентый бустинг. 

### 1. Считывание таблицы с признаками. Удаление признаков, связанных с итогами матча.

In [1]:
import pandas
raw_features = pandas.read_csv('./features.csv', index_col='match_id')

print('initial shape: '+ str(raw_features.shape))

features = raw_features.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
                              'barracks_status_radiant', 'barracks_status_dire'], axis=1)

print('new shape: '+ str(features.shape))
features.head()

initial shape: (97230, 108)
new shape: (97230, 102)


,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


### 2. Проверка выборки на наличие пропусков. Вывод названий признаков, имеющих пропуски. Представление обоснование пропуска значений этих признаков для любых 2-х из них.

In [2]:
nan_counts = features.count()
target_nan_counts = nan_counts.where(nan_counts < features.shape[0]).dropna()
print(target_nan_counts.shape[0])
target_nan_counts

12


first_blood_time               77677.0
first_blood_team               77677.0
first_blood_player1            77677.0
first_blood_player2            53243.0
radiant_bottle_time            81539.0
radiant_courier_time           96538.0
radiant_flying_courier_time    69751.0
radiant_first_ward_time        95394.0
dire_bottle_time               81087.0
dire_courier_time              96554.0
dire_flying_courier_time       71132.0
dire_first_ward_time           95404.0
dtype: float64

Больше всего пропусков наблюдается в данных, описывающих первое убийство героя одной из команд. Очевидно, не все игроки готовы в первые пять минут рисковать и нападать на противника. Атаки, особенно успешные, в основном случаются у сыгранных команд, или команд состоящих из опытных игроков, способных на хорошее взаимодействие.

### 3. Замена пропусков на 0. 

In [3]:
features = features.fillna(0)
features.count().where(nan_counts < features.shape[0]).dropna()

first_blood_time               97230.0
first_blood_team               97230.0
first_blood_player1            97230.0
first_blood_player2            97230.0
radiant_bottle_time            97230.0
radiant_courier_time           97230.0
radiant_flying_courier_time    97230.0
radiant_first_ward_time        97230.0
dire_bottle_time               97230.0
dire_courier_time              97230.0
dire_flying_courier_time       97230.0
dire_first_ward_time           97230.0
dtype: float64

### 4. Целевую переменную содержит столбец с названием radiant_win: 1, если победила команда Radiant, 0 — иначе

### 5. Обучение градиентного бустинга.

In [4]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
import datetime

time_of_training_of_30_trees = 0

tree_numbers = [5, 10, 15, 20, 25, 30, 50, 80]
kf = KFold(n_splits=5, random_state=241, shuffle=True)

x = features
y = raw_features['radiant_win']

start_time = datetime.datetime.now()

for i in tree_numbers:
    if i == 30:
        time_of_training_of_30_trees = datetime.datetime.now()
        
    
    gb = GradientBoostingClassifier(n_estimators=i, random_state=241)
    scr = cross_val_score(gb, x, y, scoring='roc_auc', cv=kf)
    score= sum(scr) / len(scr)
    print('{}: {}'.format(i, score))
    
    if i == 30:
        time_of_training_of_30_trees=datetime.datetime.now()-time_of_training_of_30_trees


print('Total training time:', datetime.datetime.now() - start_time)
print('30 trees training time:', time_of_training_of_30_trees)


%reset_selective -f kf, gb, scr

5: 0.6357376430919737
10: 0.6643877206345741
15: 0.6757420237293614
20: 0.6828535735340823
25: 0.6868481585275606
30: 0.6894962060591201
50: 0.6974548316948366
80: 0.70381673531275
Total training time: 0:08:15.320589
30 trees training time: 0:01:02.503182


Классификаторы настраивались достаточно долго. общее время настройки ~ 8 минут 10 секунд, а градиентный бустинг с 30-ю деревьями ~ 1 минуту 2 секунды. Качество классификации на 30-ти деревьях достигла 0.6895, а на 80-ти - 0.7038. Можно сделать вывод, что имеет смысл использовать больше 30 деревьев в градиентном бустинге, однако при увеличении количества деревьев производительность будет падать существенно. Для ускорения обучение можно проводить обучение по случайной подвыборке множества. Можно ограничить глубину деревьев, однако параметр нужно тщательно подбирать, чтобы это не приводило к падению качества обучения.

## Подход №2. Логистическая регрессия.

### 1. Оценка качества логистической регрессии.

In [5]:
import numpy as np
import warnings
import datetime

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

scaler = StandardScaler()
categorical_scaled_features = scaler.fit_transform(features)

x = categorical_scaled_features
y = raw_features['radiant_win']

C_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1, 2, 5, 7, 10, 15, 20, 30]

start_time = datetime.datetime.now()

kf = KFold(n_splits=5, random_state=241, shuffle=True)
for i in C_values:
    lr = LogisticRegression(penalty='l2',C=i)
    scr = cross_val_score(lr, x, y, scoring='roc_auc', cv=kf)
    score = sum(scr) / len(scr)
    print('{}: {}'.format(i, score))


print('Total training time:', datetime.datetime.now() - start_time)


%reset_selective -f kf, lr, scaler, categorical_scaled_features, scr

0.001: 0.7161802463683579
0.005: 0.7163566911760233
0.01: 0.716341462186996
0.05: 0.7163142089233235
0.1: 0.7163100836533355
0.3: 0.7163078591526911
0.5: 0.7163074013893939
0.7: 0.7163070095008914
0.9: 0.7163066324079816
1: 0.716306583645544
2: 0.7163064649978694
5: 0.7163064691888688
7: 0.7163063420811311
10: 0.7163063399602339
15: 0.7163063145103399
20: 0.7163062975560178
30: 0.7163062594232421
Total training time: 0:04:25.449988


Наилучшее качество логистической регрессии над всеми исходными признаками ~ 0.71634. Оно лучше, чем качество градиентного бустинга на 80-ти деревьях. По сравнению с 30-ю деревьями разница существенна. Дело в том, что логистическая регрессия содержит гораздо меньше весов, которые необходимо настроить, её структура проще, чем у решающих деревьев. Поэтому, она настраивается гораздо быстрее и её качество в начале обучения как правило достаточно близко к оптимуму. Из-за существенной разницы в количестве настраиваемых параметров логистическая регрессия работает сравнимо быстрее градиентного бустинга.

### 2. Оценка логистической регрессии без использования категориальных признаков в обучении.

In [6]:
import numpy as np
import warnings
import datetime

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

scaler = StandardScaler()
cleaned_features = features.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', \
                        'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
scaled_cleaned_features = scaler.fit_transform(cleaned_features)

x = scaled_cleaned_features
y = raw_features['radiant_win']

print(scaled_cleaned_features.shape)

C_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1, 2, 5, 7, 10, 15, 20, 30]

start_time = datetime.datetime.now()

kf = KFold(n_splits=5, random_state=241, shuffle=True)
for i in C_values:
    lr = LogisticRegression(penalty='l2',C=i)
    scr = cross_val_score(lr, x, y, scoring='roc_auc', cv=kf)
    score = sum(scr) / len(scr)
    print('{}: {}'.format(i, score))


print('Total training time:', datetime.datetime.now() - start_time)


%reset_selective -f kf, lr, scaler, cleaned_features, scaled_cleaned_features, scr

(97230, 91)
0.001: 0.7162355910206267
0.005: 0.7164130435225277
0.01: 0.7164009506527343
0.05: 0.7163773357189422
0.1: 0.7163737844721111
0.3: 0.7163710492777403
0.5: 0.7163706000071687
0.7: 0.7163707356064428
0.9: 0.7163706488113248
1: 0.7163707526581122
2: 0.7163706806053447
5: 0.7163705132143667
7: 0.7163704496394463
10: 0.7163704793048005
15: 0.7163704475465819
20: 0.7163704306102077
30: 0.7163704475554
Total training time: 0:04:09.560851


2. Наилучшее качество после удаления из выборки категориальных признаков ~ 0,7164. Оно чуть лучше по сравнению с предыдущем примером. Это связано с тем, что категориальный признак, будучи нормированным, является малоинформативным для классификатора, может приводить к ухудшению результата

### 3. Количество уникальных героев в игре.

In [7]:
names_to_insert = ['r', 'd']

set_of_heroes = set()
for i in range(1, 6):
    set_of_heroes = set_of_heroes.union(set(raw_features['r'+str(i)+'_hero'].unique()))
    set_of_heroes = set_of_heroes.union(set(raw_features['d'+str(i)+'_hero'].unique()))
print('max = {}'.format(max(set_of_heroes)), 'len = {}'.format(len(set_of_heroes)), sep='\n')

%reset_selective -f set_of_heroes

max = 112
len = 108


Ответ: 112

### 4. Кодирование категориальных признаков "мешком слов".

In [8]:
import numpy as np
import warnings

warnings.filterwarnings("ignore")

X_pick = np.zeros((features.shape[0], 112))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

print(X_pick.shape)

(97230, 112)


### 5. Кроссвалидация признаков на новой выборке.

In [9]:
import numpy as np
import warnings
import datetime

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

scaler = StandardScaler()
cleaned_features = features.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', \
                        'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
scaled_cleaned_features = scaler.fit_transform(cleaned_features)

print(scaled_cleaned_features.shape)

scaled_cleaned_features_with_bow = np.concatenate((scaled_cleaned_features, X_pick), axis=1)

%reset_selective -f X_pick

print(scaled_cleaned_features_with_bow.shape)

x = scaled_cleaned_features_with_bow
y = raw_features['radiant_win']

C_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1, 2, 5, 7, 10, 15, 20, 30]

start_time = datetime.datetime.now()

kf = KFold(n_splits=5, random_state=241, shuffle=True)
for i in C_values:
    lr = LogisticRegression(penalty='l2',C=i)
    scr = cross_val_score(lr, x, y, scoring='roc_auc', cv=kf)
    score = sum(scr) / len(scr)
    print('{}: {}'.format(i, score))


print('Total training time:', datetime.datetime.now() - start_time)


%reset_selective -f kf, lr, scaler, cleaned_features, scaled_cleaned_features, scr

(97230, 91)
(97230, 203)
0.001: 0.7462962333706958
0.005: 0.7512162759469865
0.01: 0.7517359377885027
0.05: 0.7519446302262013
0.1: 0.7519375046034006
0.3: 0.7519250173114229
0.5: 0.7519220537466547
0.7: 0.7519206765413137
0.9: 0.7519197080706709
1: 0.751919733264219
2: 0.7519181722092281
5: 0.7519173055919961
7: 0.7519172865150461
10: 0.751917180623139
15: 0.7519170450804328
20: 0.751917165796959
30: 0.751917237901924
Total training time: 0:07:25.368594


Наилучшее качество ~ 0.75194. Это лучше, чем достигнутые ранее результаты. Объясняется это тем, что теперь логистическая регрессия может использовать преобразованные категориальные признаки максимально информативно.

Лучшего качества модель достигает при С=0,05. Логистическая регрессия с этим параметром будет использоваться на тестовой выборке.

### 6. Построение предсказаний вероятностей победы команды Radiant для тестовой выборки с помощью логистической регрессии с преобразованием категориальных признаков.

In [10]:
import pandas
test_features = pandas.read_csv('./features_test.csv', index_col='match_id')

print('shape: '+ str(test_features.shape))

test_features.head()

shape: (17177, 102)


,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [11]:
nan_counts = test_features.count()
target_nan_counts = nan_counts.where(nan_counts < test_features.shape[0]).dropna()
print(target_nan_counts.shape[0])
target_nan_counts

12


first_blood_time               13625.0
first_blood_team               13625.0
first_blood_player1            13625.0
first_blood_player2             9411.0
radiant_bottle_time            14282.0
radiant_courier_time           17050.0
radiant_flying_courier_time    12292.0
radiant_first_ward_time        16847.0
dire_bottle_time               14335.0
dire_courier_time              17047.0
dire_flying_courier_time       12653.0
dire_first_ward_time           16914.0
dtype: float64

In [12]:
test_features = test_features.fillna(0)
test_features.count().where(nan_counts < test_features.shape[0]).dropna()

first_blood_time               17177.0
first_blood_team               17177.0
first_blood_player1            17177.0
first_blood_player2            17177.0
radiant_bottle_time            17177.0
radiant_courier_time           17177.0
radiant_flying_courier_time    17177.0
radiant_first_ward_time        17177.0
dire_bottle_time               17177.0
dire_courier_time              17177.0
dire_flying_courier_time       17177.0
dire_first_ward_time           17177.0
dtype: float64

In [13]:
names_to_insert = ['r', 'd']

set_of_heroes = set()
for i in range(1, 6):
    set_of_heroes = set_of_heroes.union(set(test_features['r'+str(i)+'_hero'].unique()))
    set_of_heroes = set_of_heroes.union(set(test_features['d'+str(i)+'_hero'].unique()))
print('max = {}'.format(max(set_of_heroes)), 'len = {}'.format(len(set_of_heroes)), sep='\n')

max = 112
len = 108


In [14]:
X_pick = np.zeros((test_features.shape[0], 112))

for i, match_id in enumerate(test_features.index):
    for p in range(5):
        X_pick[i, test_features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, test_features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

print(X_pick.shape)

(17177, 112)


In [15]:
import numpy as np
import warnings
import datetime

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
cleaned_test_features = test_features.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', \
                        'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
scaled_cleaned_test_features = scaler.fit_transform(cleaned_test_features)

print(scaled_cleaned_features.shape)

scaled_cleaned_test_features_with_bow = np.concatenate((scaled_cleaned_test_features, X_pick), axis=1)

print(scaled_cleaned_test_features_with_bow.shape)

(97230, 91)
(17177, 203)


In [16]:
warnings.filterwarnings("ignore")

x = scaled_cleaned_features_with_bow
y = raw_features['radiant_win']

print(x.shape, y.shape, sep='\n')


x_test = scaled_cleaned_features_with_bow

lr = LogisticRegression(penalty='l2',C=i)
lr.fit(x, y)

pred = lr.predict_proba(x_test)

pred

%reset_selective -f kf, lr, scaler, cleaned_test_features, scaled_cleaned_test_features, scr

(97230, 203)
(97230,)


Предсказания:

In [17]:
target_pred = [x[1] for x in pred]
for x in target_pred:
    print(x)

0.6714344983556745
0.7368763634311486
0.6038308010372452
0.2320375062109482
0.278889375275172
0.1708929041135347
0.29171381061234053
0.29732144949596145
0.22574624994241432
0.38702309785322947
0.9212424667330473
0.1534091079642786
0.1781201271606578
0.757662320559931
0.0773245306211463
0.6792352265339636
0.47408200948872997
0.4507915854584413
0.3458091885592436
0.5249918888611297
0.2762183223059895
0.900141150297187
0.8459472122973344
0.14502141359249285
0.5563248235397974
0.8036373625529537
0.615657401238632
0.7391435374111278
0.6010161560341363
0.2992192699937047
0.11664864432858486
0.4576864267407968
0.9407113886210606
0.453050710930943
0.16791725714564773
0.08144531243286438
0.8179816228204377
0.3583156854346834
0.3016304050546374
0.43765549291178885
0.8383298051319671
0.18826946468191197
0.7683596844273038
0.29999205018566316
0.11249949461602612
0.2777988816784539
0.5234275319857189
0.4755738086742744
0.49314819639691854
0.12128474744461029
0.4103082545243292
0.33034074873471064
0

0.10490091750697564
0.4532821935967241
0.23637095030415128
0.6857023527809384
0.46345393851067895
0.24834731595960435
0.44916356279936764
0.13213248301807307
0.7332270546857367
0.0798281148617621
0.13477307669678412
0.8170872224380012
0.23846858489913228
0.3689911755715115
0.666978516877291
0.7949460723075971
0.36936094044424506
0.36913615758680474
0.9452318142608169
0.22494331865011546
0.4349541320979598
0.06449383846266368
0.13118471360924794
0.6607544271663613
0.8363212730377528
0.5463878937517579
0.6345790103236504
0.516838368664151
0.4620311872087981
0.3009475769640904
0.464214526261513
0.5474695102015198
0.5737181828899263
0.49824623767639825
0.46887959216080916
0.6286467578262809
0.5311292273262634
0.4764359884634827
0.4180322649213218
0.7555411812464992
0.3623413427557656
0.2406079899152785
0.18903886759181426
0.39236585752586856
0.14636750496192252
0.3188584768735876
0.3867037266502266
0.673847891314961
0.6879821872610952
0.360592297112715
0.6074082706745868
0.7334543863404568

0.7108901865180901
0.9292842827420331
0.804738876163615
0.6806469876314653
0.6609364612067125
0.3257606988642287
0.6367294001672911
0.736143818023368
0.7965288271317623
0.5256114682717138
0.47212064714015856
0.7629692091905391
0.3202459643107093
0.7170887997461807
0.6461408323339527
0.9765251361985454
0.12360879666214665
0.8694262648366153
0.4697079466537662
0.2865772726770247
0.1527810800210883
0.5978323381951743
0.470608994001026
0.583326570797388
0.5091962515781783
0.9182954120272631
0.3971665467563686
0.642435912136262
0.08410383309150564
0.08156286631353941
0.7859473932582208
0.20243884315876423
0.3145941539670811
0.8764860787283207
0.3150766600015355
0.47613495013264284
0.5923893598160705
0.8212911376976828
0.868542384086385
0.4393474837212566
0.9049770747335135
0.42369970365750936
0.5780475885328512
0.4246219339897425
0.4961565405401314
0.8122804021623538
0.508397201648175
0.4827762894244679
0.8543590660559696
0.6500487922146811
0.5610968828240447
0.6488264581542366
0.6608445790

0.32287390665163546
0.5836246380573402
0.8264505369107259
0.638963584466153
0.21425304070275716
0.7149329805082165
0.22493360912122676
0.15644567267535858
0.5988032095832694
0.658069058857291
0.5640890978776462
0.768449277061874
0.36596658930496506
0.16083109856919914
0.4909126431104298
0.7949002629340397
0.6216407767128169
0.21425576121830892
0.6366669834172083
0.2683019908236647
0.639591553050854
0.6707238129667039
0.18332548205051674
0.5430800095011257
0.2553094470540794
0.5935212433628239
0.6857100590283485
0.35220468997274673
0.6991364438139447
0.8674767002023386
0.19193769974337865
0.7700304345390622
0.6525664589031138
0.4856745173002416
0.6884401680978135
0.699827844976514
0.7502647952268987
0.76624598106479
0.12519788989171457
0.5201293762318061
0.23913294447396738
0.46682442766808885
0.596695684584207
0.41005062158559674
0.8911391062245451
0.7905543404866636
0.7594875058494772
0.4937039567413094
0.5736930710712836
0.6423733494852079
0.4171899587089401
0.3084764235156731
0.2682

0.5148847685187898
0.5118862188301941
0.7188930864464053
0.8714136381862745
0.8929484180686984
0.4148432195297646
0.06924389395076787
0.7889172841980772
0.6738450254367999
0.5838528090468016
0.8020886603529732
0.290021371612642
0.1620634843043778
0.05892584367286231
0.9356754009574275
0.2511977310468375
0.5462922352623739
0.6406652695384449
0.5032232948215614
0.5177538944027026
0.5190287814718303
0.2209216898174558
0.5120013461868418
0.09727205927793234
0.2968578042437737
0.3031723225255121
0.19693314351278396
0.570312669825779
0.3713416942484158
0.34958706679811447
0.5552891875597811
0.4280197997699174
0.6214875135086986
0.38519063207968873
0.41485232359786195
0.4656438248190281
0.2796660080313805
0.3073352572341398
0.4032814973114358
0.3980620214070122
0.8100652734099937
0.25684469885772426
0.8984648716297208
0.8006293088305414
0.74569068912383
0.4492690537543894
0.22322198047745268
0.22679970671550678
0.5245395792232691
0.16615222784315384
0.4742271793199179
0.22881895059453503
0.88

...

0.3012257605649556
0.6869858365848821
0.3553749476546555
0.07885695270279396
0.14337206913975012
0.8806761694157499
0.4220622724011306
0.17228248577293037
0.10890824265549161
0.3878062938315486
0.3430498052902885
0.8751802352148879
0.579080502277683
0.5694337629111481
0.6405425454522505
0.7294178027573893
0.2690242938416431
0.5435890964293576
0.7301292763052779
0.8178892798260347
0.8033725685968061
0.5024874326967836
0.6904504214063153
0.658815621474042
0.9681890033088639
0.4592809658008525
0.5851785198763385
0.9016501124566653
0.4079476008917256
0.6949503728690789
0.41329081066319123
0.5287333138748075
0.7914982500736311
0.641124594863835
0.1448159860818488
0.4398809039849956
0.666987883579767
0.3255142830188318
0.4325238283852857
0.6357737178325528
0.6799307405970698
0.8589942016698946
0.20634299372454426
0.780787647913101
0.7090699433449164
0.3340102286005176
0.41374011134525385
0.7392705982964233
0.7176913984794775
0.5446227096008074
0.20223523787814887
0.3902384825877919
0.6240298

0.38456856231185527
0.568002274326891
0.518967270028365
0.6509566674137174
0.8113776341488794
0.21926416353974706
0.48392405694881835
0.50203208933244
0.9018268845581867
0.4061952713637023
0.1464238736942739
0.7151334732952112
0.24051496806358927
0.5940040008176404
0.2502687012720748
0.15664228794796659
0.3919925672031776
0.416322821688924
0.6680430828858203
0.7081276553770631
0.357197191994841
0.2697772803498965
0.7530612630721517
0.4983324759668294
0.5537444114198409
0.8328235585528652
0.7101265247273062
0.2640304649866843
0.716633606044134
0.6367905175211998
0.7374823666675338
0.7319856826075687
0.6489096375888618
0.179270953119934
0.21696135590788088
0.7962833697063215
0.713150076776459
0.5260827820599573
0.4904800517818956
0.631275360768546
0.4845285119848496
0.47852761703451663
0.439879751250756
0.3417274559766069
0.28115946174696005
0.6999219982932363
0.6819187083947679
0.36320364291899143
0.4684199387251039
0.5624906606356173
0.18810006351112507
0.5276345781061161
0.48036292295

0.8763485686815112
0.8877006292731855
0.7832192618258581
0.7041275413956128
0.40251697979542966
0.887704241147089
0.647213305747573
0.859438164797089
0.845924261833256
0.3495792563716056
0.4479325188169248
0.5337857014973355
0.4359185512331221
0.19494528853099624
0.6719297364734005
0.692912619409393
0.8687280068592521
0.5741127348621363
0.6502479318514661
0.7147046593303225
0.9300094695889288
0.689603283911452
0.8945003818980476
0.525613079295831
0.3911235913249208
0.29539538008905347
0.08517762251819083
0.642260263115669
0.4390849551765285
0.4645017359877127
0.5981354504025586
0.47547307944430556
0.8039010661634394
0.5385867738083411
0.7245232926759168
0.26271376202665486
0.5732583094963329
0.14045286170791246
0.36564903616441863
0.5038492882294957
0.16842461981573145
0.5510155682176151
0.525803165392815
0.4145904840501634
0.7444415055099555
0.5669739942153895
0.8494727004205193
0.4214745126796896
0.3398464055323615
0.561186782239766
0.3972410665909984
0.5185212494291488
0.11752442908

0.6312821966687037
0.6699264190355417
0.8537827649552546
0.3500386044250422
0.49096797322848884
0.5096326057266434
0.22786182189334503
0.6335418027576357
0.8101771706168314
0.2908106029297078
0.4650910135068965
0.41619911037836366
0.321883771855643
0.6586791676501985
0.811878329587707
0.8458725612454019
0.545185902820311
0.6970999903237965
0.6773558720317925
0.8094827983284804
0.8142513623165806
0.7657755869964531
0.8163838070878132
0.5049398344760068
0.565921680802592
0.5787920355821508
0.3148052446555941
0.28831380441358234
0.8090664376463278
0.42239221298482066
0.5659535091689484
0.7118077991345417
0.399737133600989
0.9198135968552744
0.9527714096369196
0.9792485886686902
0.3594055410676891
0.3908566879010138
0.7566591753667121
0.5798946649936016
0.32231047620640063
0.2418525020549025
0.5883746355296621
0.731954862781647
0.5081692740491346
0.43401764516053165
0.2200851125015836
0.760231533072204
0.6376886812265908
0.7394796547271449
0.9229422650107056
0.6575192816183406
0.7678875042

0.3221073098661499
0.7403750106386301
0.7437379624837621
0.6486489569251642
0.3469162410417536
0.33382571543331524
0.5297660991802682
0.4112587310620644
0.6325503668355041
0.07318915660029114
0.8874610575130807
0.32167599475063385
0.5695927382790343
0.5526247239022726
0.5866665112563976
0.16936086878565657
0.44642649025795295
0.6315985437080853
0.33811752699786823
0.4060388491225201
0.5369864453108025
0.29360043595993757
0.41228331795469175
0.4099859716795111
0.7144836905476366
0.31418947961955274
0.1799383956811495
0.45731762206560506
0.6013509024470428
0.7667183965341758
0.8046556558101627
0.5621891799569684
0.5474544268628717
0.4448582027661109
0.6999335343681631
0.29582958109832636
0.644673021175748
0.943501519061109
0.6002126417540865
0.36343283070578936
0.5829469792337495
0.3694472513377672
0.41592793887414414
0.5929637863692472
0.910088869733086
0.861533785314567
0.6177202695830724
0.62106617857538
0.6731924343874731
0.09384653010118718
0.7769550137890789
0.48141992320140853
0.3

In [18]:
print('min: {} \nmax: {}'.format(min(target_pred), max(target_pred)))

min: 0.00132093576194957 
max: 0.9987355308409461


Min: 0.00132093576194957

Max: 0.9987355308409461